# Analyze the data file generated by moltemplate
**fsdfs**
<br>
<br>
ffd

## Conversion between RB-style dihedral function and Fourier (opls) style

In [2]:


def opls2RB(Flist):
    # convert the LAMMPS's "opls" dihedral form to RB form which is used in Gromacs
    # v_opls = f1*(1+cos(fi))/2 + f2*(1-cos(2fi))/2 + f3*(1+cos(3fi))/2 + f4*(1-cos(4fi))/2
    f0 = 0
    f1 = Flist[0]
    f2 = Flist[1]
    f3 = Flist[2]
    c0 = 0.5*(f1+2*f2+f3)+f0
    c1 = -0.5*(f1-3*f3)
    c2 = -f2
    c3 = -2*f3

    return [c0,c1,c2,c3]    

def RB2opls(RBlist):
    # convert RB (or multi/harmonic) dihedral form to LAMMPS's opls form
    # v_opls = f1*(1+cos(fi))/2 + f2*(1-cos(2fi))/2 + f3*(1+cos(3fi))/2
    # v_RB = c0 + c1*cos(fi) + c2*cos(fi)^2 + c3*cos(fi)^3
    [c0,c1,c2,c3] = RBlist
    f3 = -0.5*c3
    f2 = -c2
    f1 = -1.5*c3-2*c1
    
    return [f1,f2,f3,0]

# # this function has been depreciated
# # because RB can only converted to the Fourier function with a constant term
# # see my dicussion in Onenote
# def Fourier2RB(Flist):
#     # lammps standard opls style dihedral:
#     # v_dihedral = f1*(1+cos(fi))/2 + f2*(1-cos(2fi))/2 + f3*(1+cos(3fi))/2
# #     Flist = [f1,f2,f3,f4]
#     [f1,f2,f3,f4] = Flist
#     c0 = f2 + (f1+f3)/2.0
#     c1 = 0.5 * (-1.0*f1+3*f3)
#     c2 = -1.0*f2 + 4*f4
#     c3 = -2 * f3

#     return [c0,c1,c2,c3]



def Fourier2RB_Siu(Flist):
    # lopls paper by Siu et al. 2012
    # v_Fourier = f0 + f1*(1+cos(fi))/2 + f2*(1-cos(2fi))/2 + f3*(1+cos(3fi))/2
    # v_RB = c0 + c1*cos(fi) + c2*cos(fi)^2 + c3*cos(fi)^3
    #
    [f0,f1,f2,f3] = Flist
    c0 = 0.5*(f1+2*f2+f3)+f0
    c1 = -0.5*(f1-3*f3)
    c2 = -f2
    c3 = -2*f3

    return [c0,c1,c2,c3]


# # this function has been depreciated
# # because RB can only converted to the Fourier function with a constant term
# # see my dicussion in Onenote
# def RB2Fourier(RBlist):
#     # convert RB dihedral form to lammps's opls standard form
#     # RB: v_RB = c0 + c1*cos(fi) + c2*cos(fi)^2 + c3*cos(fi)^3
#     [c0,c1,c2,c3] = RBlist

#     f3 = -0.5*c3
#     f2 = -c2
#     f1 = -1.5*c3-2*c1

#     return [f1,f2,f3,0]


def RB2Fourier_Siu(RBlist):
    # convert RB dihedral form to Siu 2012's opls form
    # lopls paper by Siu et al. 2012
    # v_Fourier = f0 + f1*(1+cos(fi))/2 + f2*(1-cos(2fi))/2 + f3*(1+cos(3fi))/2
    # v_RB = c0 + c1*cos(fi) + c2*cos(fi)^2 + c3*cos(fi)^3
    [c0,c1,c2,c3] = RBlist
    f3 = -0.5*c3
    f2 = -c2
    f1 = -1.5*c3-2*c1
    f0 = c0 - 0.5*(f1+2*f2+f3)
    
    return [f0,f1,f2,f3]

## Unit conversion

In [3]:
def kcal2kj (value):
    """
    convert unit from kcal/mol to kJ/mol 
    1 kcal = 4.184 kJ
    """

    return (float(value)*4.184)


def kj2kcal (value):
    """
    convert unit from kj/mol to kcal/mol 
    1 kcal = 4.184 kJ
    """
    
    return (float(value)/4.184)


def ev2kj (value):
    """
    convert unit from eV to kJ/mol 
    1 eV = 96 kJ/mol
    """
    
    return float(value)*96

In [4]:
with open("system.in.settings") as file:
    data_coeff = file.readlines()

for line in data_coeff:
    b = line.split()
    if b[0] == "dihedral_coeff":
        d_para = RB2opls([float(i) for i in b[2:6]])
        print(f"dihedral_coeff {b[1]} {d_para[0]:7.5f} {d_para[1]:7.5f} {d_para[2]:7.5f} {d_para[3]:7.5f}")

dihedral_coeff 1 -0.27700 1.22800 -0.69400 0.00000
dihedral_coeff 2 -0.00000 -0.00000 -0.00000 0.00000
dihedral_coeff 3 0.00000 -0.00000 -0.55300 0.00000
dihedral_coeff 4 -0.00000 -0.00000 0.13200 0.00000
dihedral_coeff 5 -0.00000 5.12400 -0.00000 0.00000
dihedral_coeff 6 4.66900 5.12400 -0.00000 0.00000
dihedral_coeff 7 -1.69700 -0.45600 0.58500 0.00000
dihedral_coeff 8 0.00001 -0.00000 -0.07600 0.00000
dihedral_coeff 9 1.30000 -0.05000 0.20000 0.00000
dihedral_coeff 10 -0.00000 -0.00000 0.30000 0.00000
dihedral_coeff 11 -0.00000 -0.00000 0.30000 0.00000
dihedral_coeff 12 -1.22000 -0.12600 0.42200 0.00000
dihedral_coeff 13 -0.00000 -0.00000 0.19800 0.00000
dihedral_coeff 14 1.71100 -0.50000 0.66300 0.00000


In [5]:
with open("system_lopls.in.settings") as file:
    data_coeff = file.readlines()

for line in data_coeff:
    b = line.split()
    if b[0] == "dihedral_coeff":
        d_para = RB2opls([float(i) for i in b[2:6]])
        print(f"dihedral_coeff {b[1]} {d_para[0]:7.5f} {d_para[1]:7.5f} {d_para[2]:7.5f} {d_para[3]:7.5f}")

dihedral_coeff 1 -0.00000 -0.00000 -0.00000 0.00000
dihedral_coeff 2 -0.00000 0.00000 0.30000 0.00000
dihedral_coeff 3 -0.00000 0.00000 0.30000 0.00000
dihedral_coeff 4 0.00000 0.00000 0.19800 0.00000
dihedral_coeff 5 0.64470 -0.21434 0.17822 0.00000
dihedral_coeff 6 3.11924 5.73771 -0.00000 0.00000
dihedral_coeff 7 -0.00000 5.73772 -0.00000 0.00000
dihedral_coeff 8 -0.00742 0.00218 0.11181 0.00000
dihedral_coeff 9 -1.73540 -1.24844 0.62389 0.00000
dihedral_coeff 10 0.88498 -0.62690 -0.49334 0.00000
dihedral_coeff 11 -0.27601 1.23685 -0.67075 0.00000
dihedral_coeff 12 -0.00213 0.01735 -0.29521 0.00000
dihedral_coeff 13 -2.30739 -0.62733 0.62195 0.00000
dihedral_coeff 14 2.25871 -1.02408 1.00710 0.00000
